<a href="https://colab.research.google.com/github/roytalari/SparkPySpark/blob/main/Lab3pySparkUDF%26Joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Spark
## Set Environment variables
## Create a Spark Session

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.6.tgz
!tar -xvf spark-2.4.3-bin-hadoop2.6.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.6"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Test Spark with dummy DF

In [3]:
df = spark.createDataFrame([{"Google": "Colab","Spark":"Scala"},{"Google": "Runbook","Spark":"PySpark"}])
df.show()

/content/spark-2.4.3-bin-hadoop2.6/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-------+-------+
| Google|  Spark|
+-------+-------+
|  Colab|  Scala|
|Runbook|PySpark|
+-------+-------+



## COPY A DATA FILE TO YOUR LOCAL COLAB ENVIRONMENT

In [4]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/store_customers.csv

--2020-10-24 23:46:07--  https://raw.githubusercontent.com/futurexskill/bigdata/master/store_customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 199132 (194K) [text/plain]
Saving to: ‘store_customers.csv’

store_customers.csv 100%[===================>] 194.46K  --.-KB/s    in 0.05s   

2020-10-24 23:46:07 (4.02 MB/s) - ‘store_customers.csv’ saved [199132/199132]



## Check if the files are copied

In [5]:
!ls

sample_data		   spark-2.4.3-bin-hadoop2.6.tgz
spark-2.4.3-bin-hadoop2.6  store_customers.csv


# bring the store_transaction file to your environment

In [6]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/store_transactions.csv

--2020-10-24 23:48:58--  https://raw.githubusercontent.com/futurexskill/bigdata/master/store_transactions.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7257374 (6.9M) [text/plain]
Saving to: ‘store_transactions.csv’

store_transactions. 100%[===================>]   6.92M  15.5MB/s    in 0.4s    

2020-10-24 23:48:58 (15.5 MB/s) - ‘store_transactions.csv’ saved [7257374/7257374]



In [7]:
!ls

sample_data		       store_customers.csv
spark-2.4.3-bin-hadoop2.6      store_transactions.csv
spark-2.4.3-bin-hadoop2.6.tgz


Create DataFrames 

In [9]:
customerDF = spark.read.csv("store_customers.csv", header= True)

# display data in DF

In [10]:
customerDF.show()

+----------+---+------+------+-------+
|CustomerID|Age|Salary|Gender|Country|
+----------+---+------+------+-------+
|         1| 72| 20000|  Male|Germany|
|         2| 72| 22000|Female| France|
|         3| 70| 24000|Female|England|
|         4| 75|  2600|  Male|England|
|         5| 33| 50000|  Male| France|
|         6| 52| 35000|Female|England|
|         7| 31|  4300|  Male|Germany|
|         8| 37| 32000|Female| France|
|         9| 76| 35000|  Male|Germany|
|        10| 58| 37000|Female| France|
|        11| 70| 25000|  Male|Germany|
|        12| 28| 27000|Female| France|
|        13| 21| 29000|Female|England|
|        14| 34|  7600|  Male|England|
|        15| 45| 55000|  Male| France|
|        16| 32| 40000|Female|England|
|        17| 62|  9300|  Male|Germany|
|        18| 54| 37000|Female| France|
|        19| 33| 40000|  Male|Germany|
|        20| 46| 42000|Female| France|
+----------+---+------+------+-------+
only showing top 20 rows



# count shows customer has 7K rows

In [13]:
customerDF.count()

7000

# Create DF for transaxtions

In [14]:
trasactionDF = spark.read.csv("store_transactions.csv", header=True)

In [15]:
trasactionDF.show()

+----------+---------+------+----------+
|CustomerID|ProductID|Amount|      Date|
+----------+---------+------+----------+
|      3427|        3|  7541|22-11-2019|
|      4378|       14|  7271|15-12-2019|
|      3751|       47|  4276|20-11-2019|
|      6899|      146|  8923|22-11-2019|
|      4561|       46|  4891|30-11-2019|
|      2299|      143|  7545|05-12-2019|
|       553|       43|  1147|08-12-2019|
|      3406|      134|   245|12-12-2019|
|      5278|       34|  8765|09-12-2019|
|      2456|       68|  3820|30-11-2019|
|      6963|       58|  1850|27-11-2019|
|      4700|      140|  3948|27-11-2019|
|      3566|       76|   401|17-12-2019|
|      2212|       38|  4750|16-12-2019|
|      4677|       18|  1149|18-11-2019|
|      4295|       35|  3241|19-11-2019|
|      4228|       91|  6405|01-12-2019|
|      2466|       83|  2978|10-12-2019|
|      3136|       67|  6581|28-11-2019|
|      6563|       80|  2878|21-11-2019|
+----------+---------+------+----------+
only showing top

# Transactions has over 1 millions rows

In [16]:
trasactionDF.count()

1048575

# Extracting year from date field in customerDF using UDF


In [18]:
from pyspark.sql.functions import udf

In [34]:
trasactionDF.select("date").distinct().show()

+----------+
|      date|
+----------+
|13-12-2019|
|18-11-2019|
|      null|
|17-12-2019|
|22-11-2019|
|06-12-2019|
|26-11-2019|
|27-11-2019|
|08-12-2019|
|12-12-2019|
|03-12-2019|
|09-12-2019|
|28-11-2019|
|16-12-2019|
|30-11-2019|
|20-11-2019|
|15-12-2019|
|23-11-2019|
|10-12-2019|
|24-11-2019|
+----------+
only showing top 20 rows



In [35]:
extract_year = udf(lambda date: date.split('-')[2])

In [37]:
trasactionDF  = trasactionDF.withColumn("Year",extract_year(trasactionDF.Date))

In [38]:
trasactionDF.show()

+----------+---------+------+----------+----+
|CustomerID|ProductID|Amount|      Date|Year|
+----------+---------+------+----------+----+
|      3427|        3|  7541|22-11-2019|2019|
|      4378|       14|  7271|15-12-2019|2019|
|      3751|       47|  4276|20-11-2019|2019|
|      6899|      146|  8923|22-11-2019|2019|
|      4561|       46|  4891|30-11-2019|2019|
|      2299|      143|  7545|05-12-2019|2019|
|       553|       43|  1147|08-12-2019|2019|
|      3406|      134|   245|12-12-2019|2019|
|      5278|       34|  8765|09-12-2019|2019|
|      2456|       68|  3820|30-11-2019|2019|
|      6963|       58|  1850|27-11-2019|2019|
|      4700|      140|  3948|27-11-2019|2019|
|      3566|       76|   401|17-12-2019|2019|
|      2212|       38|  4750|16-12-2019|2019|
|      4677|       18|  1149|18-11-2019|2019|
|      4295|       35|  3241|19-11-2019|2019|
|      4228|       91|  6405|01-12-2019|2019|
|      2466|       83|  2978|10-12-2019|2019|
|      3136|       67|  6581|28-11